In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import geopy

import geopandas as gpd
from shapely.geometry import Point

In [55]:
#La ou est située le fichier train.csv sur votre ordinateur 
path = "/Users/gilles/Documents/Entretiens_data_2020/My_Traffic/Data/"

In [56]:
#train_sample =pd.read_csv(path+"train_sample.csv", index_col = 0 )
#train_sample.head()

In [57]:
pd.read_csv?

In [58]:
train = pd.read_csv(path+"train.csv")
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [59]:
np.shape(train)

(1458644, 11)

In [60]:
#train.to_csv("train_sample.csv")

## 1. Etude préalable

In [61]:
train.trip_duration.head()

0     455
1     663
2    2124
3     429
4     435
Name: trip_duration, dtype: int64

In [62]:
train.trip_duration.describe()

count    1.458644e+06
mean     9.594923e+02
std      5.237432e+03
min      1.000000e+00
25%      3.970000e+02
50%      6.620000e+02
75%      1.075000e+03
max      3.526282e+06
Name: trip_duration, dtype: float64

In [63]:
#On considère qu'un trajet ne peut pas durer plus de 24h
duree_max = 24*60*60
print(duree_max)

86400


## 2. Réponses aux question

On cherche à calculer les indicateurs ci-dessous :
- la vitesse moyenne de chaque trajet,
- le nombre de trajets effectués en fonction du jour de la semaine,
- le nombre de trajets effectués en fonction de l’horaire de la journée par tranche de 4h,
- le nombre de km parcourus par jour de la semaine.

### 2.1 Vitesse moyenne de chaque trajet

Pour avoir la vitesse moyenne de chaque trajet, il nous faut calculer la distance de chaque trajet à partir des coordonnées de départ et d'arrivée.

On va estimer à la baisse cette distance en calculant seulement la distance à vol d'oiseau entre le point de départ et le point d'arrivée. Pour ce faire on va utiliser la distance haversine.

#### 2.1.1 Calcul de la distance (au moins 2min)

In [64]:
#Il s'agit de la distance great cricle / alternative à la geodesic 
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    
    return np.round(res,5)*1000

In [65]:
haversine_distance(train["pickup_latitude"].iloc[0], train["pickup_longitude"].iloc[0], train["dropoff_latitude"].iloc[0], train["dropoff_longitude"].iloc[0] )

1498.52

In [66]:
def speed_estimate(df):
    
    #On crée d'abord la variable distance
    df["distance"] = df.apply(lambda x: haversine_distance(x["pickup_latitude"], x["pickup_longitude"], x["dropoff_latitude"], x["dropoff_longitude"]), axis = 1)
    df["speed"] = (df["distance"] / df["trip_duration"]) * 3.6
    
    return df

In [68]:
train = speed_estimate(train)
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,speed
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1498.52,11.856422
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1805.51,9.803674
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6385.10,10.822203
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1485.50,12.465734
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1188.59,9.836607


In [69]:
train.speed.describe()

count    1.458644e+06
mean     1.442321e+01
std      1.497752e+01
min      0.000000e+00
25%      9.121284e+00
50%      1.279200e+01
75%      1.784488e+01
max      9.274834e+03
Name: speed, dtype: float64

### 2.2 Nombre de trajets en fonction du jour de la semaine

In [70]:
#import calendar
import datetime

In [71]:
#ON se base plutot sur Pickup datetime pour le nombre de trajets

In [1]:
type(train.iloc[0]["pickup_datetime"])

NameError: name 'train' is not defined

In [74]:
#train["pickup_datetime"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if type(x) == str else x)

In [34]:
ISOWeekDays = ("NoZeroInISOWeek","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")

ISOWeekDays[date_time_obj.isoweekday()]

'Monday'

In [75]:
def weekday(df):
    
    ISOWeekDays = ("NoZeroInISOWeek","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    
    df["weekday"] = df["pickup_datetime"].apply(lambda x: ISOWeekDays[datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').isoweekday()] if type(x) == str else x)
    
    return df 
    

In [76]:
train = weekday(train)
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,speed,weekday
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1498.52,11.856422,Monday
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1805.51,9.803674,Sunday
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6385.10,10.822203,Tuesday
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1485.50,12.465734,Wednesday
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1188.59,9.836607,Saturday


In [78]:
train.weekday.value_counts()

Friday       223533
Saturday     220868
Thursday     218574
Wednesday    210136
Tuesday      202749
Sunday       195366
Monday       187418
Name: weekday, dtype: int64

### 2.3 Nombre de trajets en fonction de l'horaire de la journée (par tranche de 4h)



In [79]:
train["hour_4"] = pd.cut?

In [85]:
date_time_obj = datetime.datetime.strptime(train.iloc[0]["pickup_datetime"], '%Y-%m-%d %H:%M:%S')
date_time_obj.hour

17

In [81]:
def check_hour(x):
    if 0 <= x < 4:
        return '0-4'
    elif 4 <= x < 8:
        return '4-8'  
    elif 8 <= x < 12:
        return '8-12'
    elif 12 <= x < 16:
        return '12-16'
    elif 16 <= x < 20:
        return '16-20'
    else:
        return '20-24'

In [82]:
def hour(df):
    
    df["hour"] =  df["pickup_datetime"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour if type(x) == str else x)
    
    return (df)

In [83]:
train = hour(train)
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,speed,weekday,hour
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1498.52,11.856422,Monday,17
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1805.51,9.803674,Sunday,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6385.10,10.822203,Tuesday,11
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1485.50,12.465734,Wednesday,19
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1188.59,9.836607,Saturday,13


In [84]:
#train["hour_4"] = train["hour"].apply(lambda x: check_hour(x))

In [96]:
[i for i in range(0, 25,4)]

[0, 4, 8, 12, 16, 20, 24]

In [98]:
pd.cut?

In [104]:
#Rajouter le include_lowest
train["hour_tranche"] = pd.cut(train["hour"],[i for i in range(0,25,4)], include_lowest = True)

In [100]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance,speed,weekday,hour,hour_tranche
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1498.52,11.856422,Monday,17,"(16.0, 20.0]"
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1805.51,9.803674,Sunday,0,"(-0.001, 4.0]"
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6385.10,10.822203,Tuesday,11,"(8.0, 12.0]"
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1485.50,12.465734,Wednesday,19,"(16.0, 20.0]"
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1188.59,9.836607,Saturday,13,"(12.0, 16.0]"


In [103]:
train.hour_tranche.value_counts().sum()

1458644

In [90]:
train.hour.value_counts()

18    90600
19    90308
21    84185
20    84072
22    80492
17    76483
14    74292
12    71873
15    71811
13    71473
23    69785
11    68476
9     67663
8     67053
10    65437
16    64313
7     55600
0     53248
1     38571
6     33248
2     27972
3     20895
4     15792
5     15002
Name: hour, dtype: int64

In [89]:
train.isnull().sum()

id                        0
vendor_id                 0
pickup_datetime           0
dropoff_datetime          0
passenger_count           0
pickup_longitude          0
pickup_latitude           0
dropoff_longitude         0
dropoff_latitude          0
store_and_fwd_flag        0
trip_duration             0
distance                  0
speed                     0
weekday                   0
hour                      0
hour_tranche          53248
dtype: int64

### 2.4 Nombre de km parcourus par jour de la semaine

In [105]:
# Il faut à la fois récupérer la distance parcourue et le jour de la semaine
train.groupby("weekday")["distance"].sum()/1000

weekday
Friday       758724.47103
Monday       668482.11123
Saturday     736411.13072
Sunday       726453.24973
Thursday     747677.62833
Tuesday      678328.09510
Wednesday    702918.79603
Name: distance, dtype: float64

La limitation est que si un trajet a commencé juste avant minuit et qu'il s'est arrêté après minuit, je compte toutes la distance parcoure pour le jour d'avant 

In [109]:
#On enregistre le dataset train auquel des variables ont été ajouté 
train.to_csv("featured_train.csv")

In [108]:
 train.to_csv?